### ____init____ialisieren der benötigten Klassen
Im nächsten Schritt werden die Klassen definiert, deren Objekte in der CSV beschrieben werden.

In [212]:
class Task:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Activity:
    def __init__(self, ID, name, parentTask, semaphoresIN, semaphoresOUT, mutexes):
        self.ID = ID
        self.name = name
        self.parentTask = parentTask
        self.semaphoresIN = semaphoresIN
        self.semaphoresOUT = semaphoresOUT
        self.mutexes = mutexes

class Mutex:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Semaphore:
    def __init__(self, ID, name, groupWith, initialValue, activityIN, activityOUT):
        self.ID = ID
        self.name = name
        self.groupWith = groupWith
        self.initialValue = initialValue
        self.activityIN = activityIN
        self.activityOUT = activityOUT

### Test der Klassen
Um die Klassen zu testen wird das folgende Diagramm aufgebaut: 

![Fehler beim Abrufen der Datei: ./img/test_diagram.png](./img/test_diagram.png)

In [ ]:
task1 = Task(1, "Task1", [1])
task2 = Task(2, "Task2", [2])
task3 = Task(3, "Task3", [3])
task4 = Task(4, "Task4", [4])
task5 = Task(5, "Task5", [5, 7])
task6 = Task(6, "Task6", [6])

activity1 = Activity(1, "Activity1", 1, [12, 13], [])
activity2 = Activity(2, "Activity2", 2, [24], [234])
activity3 = Activity(3, "Activity3", 3, [36], [234])
activity4 = Activity(4, "Activity4", 4, [46], [234])
activity5a = Activity(5, "Activity5a", 5, [55], [])
activity5b = Activity(7, "Activity5b", 5, [56], [])
activity6 = Activity(6, "Activity6", 6, [65], [])

mutex234 = Mutex(234, "Mutex234", [2, 3, 4])

semaphore12 = Semaphore(12, "Semaphore12", [2], None)
semaphore13 = Semaphore(13, "Semaphore13", [3], None)
semaphore24 = Semaphore(24, "Semaphore24", [4], None)
semaphore36 = Semaphore(36, "Semaphore36", [6], None)
semaphore46 = Semaphore(46, "Semaphore46", [6], None)
semaphore55 = Semaphore(55, "Semaphore55", [7], 1)
semaphore56 = Semaphore(56, "Semaphore56", [5], None)
semaphore65 = Semaphore(65, "Semaphore65", [5], 1)

### Einlesen der CSV Datei
Als nächstes sollen die Objekte aus einer CSV Datei eingelesen werden.

In [213]:
dataFilePath = "./dataTest.csv"

data = []
linecells = []

import pandas as pd
df = pd.read_csv(dataFilePath, sep=',')

In [214]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,None,12;13;16,Semaphore12;Semaphore13;Semaphore16,None;None;None,51,None,None
1,2,Task2,2,Activity2,None,24,Semaphore24,None,12,234,Mutex234
2,3,Task3,3,Activity3,None,36,Semaphore36,None,13,234,Mutex234
3,4,Task4,4,Activity4,None,46,Semaphore46,None,24,234,Mutex234
4,5,Task5,5,Activity5a,None,55,Semaphore55,None,56;65,None,None
5,5,Task5,7,Activity5b,None,51;56,Semaphore51;Semaphore56,1;1,55,None,None
6,6,Task6,6,Activity6,None,65,Semaphore65,None,[36;46];16,None,None


In [215]:
# split seperated values into list elements
if not isinstance(df["Semaphore_ID"][0], list):
    df["Semaphore_ID"] = df["Semaphore_ID"].str.split(";")
if not isinstance(df["Semaphore_Name"][0], list):
    df["Semaphore_Name"] = df["Semaphore_Name"].str.split(";")
if not isinstance(df["Semaphore_Initial_Value"][0], list):
    df["Semaphore_Initial_Value"] = df["Semaphore_Initial_Value"].str.split(";")
if not isinstance(df["Predecessor_Semaphore_ID"][0], list):
    df["Predecessor_Semaphore_ID"] = df["Predecessor_Semaphore_ID"].str.split(";")
if not isinstance(df["Mutex_ID"][0], list):
    df["Mutex_ID"] = df["Mutex_ID"].str.split(";")
if not isinstance(df["Mutex_Name"][0], list):
    df["Mutex_Name"] = df["Mutex_Name"].str.split(";")

In [216]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,None,"[12, 13, 16]","[Semaphore12, Semaphore13, Semaphore16]","[None, None, None]",[51],[None],[None]
1,2,Task2,2,Activity2,None,[24],[Semaphore24],[None],[12],[234],[Mutex234]
2,3,Task3,3,Activity3,None,[36],[Semaphore36],[None],[13],[234],[Mutex234]
3,4,Task4,4,Activity4,None,[46],[Semaphore46],[None],[24],[234],[Mutex234]
4,5,Task5,5,Activity5a,None,[55],[Semaphore55],[None],"[56, 65]",[None],[None]
5,5,Task5,7,Activity5b,None,"[51, 56]","[Semaphore51, Semaphore56]","[1, 1]",[55],[None],[None]
6,6,Task6,6,Activity6,None,[65],[Semaphore65],[None],"[[36, 46], 16]",[None],[None]


In [217]:
# only during the creation process of the tree
tasks_IDs = []
tasks = []

activities_IDs = []
activities = []

semaphore_IDs = []
semaphores = []

mutex_IDs = []
mutexs = []

for index, row in df.iterrows():
    # first: instantiation of objects
    if row["Task_ID"] not in tasks_IDs:
        tasks_IDs.append(row["Task_ID"])
        tasks.append(Task(row["Task_ID"], row["Task_Name"], []))
    
    if row["Activity_ID"] not in activities_IDs:
        activities_IDs.append(row["Activity_ID"])
        activities.append(Activity(row["Activity_ID"], row["Activity_Name"], [], row["Predecessor_Semaphore_ID"], [], []))
        print(row["Predecessor_Semaphore_ID"])
    
    # semaphoren anhand der VorgängerID erstellen, nicht anhand der herausgehenden ID
    for index, semaphore_ID in enumerate(row["Semaphore_ID"]):
        if not semaphore_ID == "None" and semaphore_ID not in semaphore_IDs:
            semaphore_IDs.append(semaphore_ID)
            semaphores.append(Semaphore(semaphore_ID, row["Semaphore_Name"][index], [], row["Semaphore_Initial_Value"][index], [], []))
        
    for index, mutex_ID in enumerate(row["Mutex_ID"]):
        if not mutex_ID == "None" and mutex_ID not in mutex_IDs:
            mutex_IDs.append(mutex_ID)
            mutexs.append(Mutex(mutex_ID, row["Mutex_Name"][index], []))
            
    # add semaphoreIN to activities and semaphores
    
    # secondly: linking of objects
    tasks[tasks_IDs.index(row["Task_ID"])].activities.append(activities[activities_IDs.index(row["Activity_ID"])])
    
    activities[activities_IDs.index(row["Activity_ID"])].parentTask = tasks[tasks_IDs.index(row["Task_ID"])]
    for semaphore_ID in row["Semaphore_ID"]:
        if not semaphore_ID == "None":
            activities[activities_IDs.index(row["Activity_ID"])].semaphoresOUT.append(semaphores[semaphore_IDs.index(semaphore_ID)])
            semaphores[semaphore_IDs.index(semaphore_ID)].activityOUT = activities[activities_IDs.index(row["Activity_ID"])]
    for mutex_ID in row["Mutex_ID"]:
        if not mutex_ID == "None":
            activities[activities_IDs.index(row["Activity_ID"])].mutexes.append(mutexs[mutex_IDs.index(mutex_ID)])
            mutexs[mutex_IDs.index(mutex_ID)].activities.append(activities[activities_IDs.index(row["Activity_ID"])])


['51']
['12']
['13']
['24']
['56', '65']
['55']
['[36', '46]', '16']


In [218]:
# add semaphoreIN to activities and semaphores
for activity in activities:
    buf = activity.semaphoresIN
    activity.semaphoresIN = []
    while len(buf) > 0:
        semaphore_id = buf.pop(0)
        # if start equals '[' -> start of or)
        if semaphore_id[0] == '[':
            semaphore_id = semaphore_id[1:]
            
            if semaphore_id[-1] == ']':
                semaphore_id = semaphore_id[:-1]
                activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
                semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
                print(semaphore_id)
            else:
                semaphore_ids = []
                while semaphore_id[-1] != ']':
                    semaphore_ids.append(semaphore_id)
                    semaphore_id = buf.pop(0)
                semaphore_ids.append(semaphore_id[:-1])
                for semaphore_id in semaphore_ids:
                    activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
                    semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
                    # set the groupWith value of the semaphores and add every semaphore to the group except for the current one
                    for semaphore in semaphore_ids:
                        if not semaphore == semaphore_id:
                            semaphores[semaphore_IDs.index(semaphore_id)].groupWith.append(semaphores[semaphore_IDs.index(semaphore)])
                print(semaphore_ids)
        
        # if end equals ']' -> end of or)
        
        # else -> and
        else:
            activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
            semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
            print(semaphore_id)

51
12
13
24
56
65
55
['36', '46']
16


In [56]:
root = tasks[0]
print("hallo")

hallo


In [68]:
print("Task 1 ist mit folgendem Task verbunden: Task " + str(tasks[0].activities[0].semaphoresOUT[0].activityIN.parentTask.ID))

Task 1 ist mit folgendem Task verbunden: Task 2


In [2]:
import graphviz as gv

dot = gv.Digraph(comment='Rectangle with Arrow')

dot.node('A', shape='rectangle', label='Rectangle A')
dot.node('B', shape='rectangle', label='Rectangle B')

dot.edge('A', 'B', arrowhead='vee')

dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg.jpg'

In [3]:
dot = gv.Digraph(comment='Dashed Arrow')

dot.node('A', shape='rectangle', label='Rectangle A', style='filled', fillcolor='lightblue')
dot.node('B', shape='rectangle', label='Rectangle B', style='filled', fillcolor='lightgreen')

dot.edge('A', 'B', arrowhead='vee', style='dashed')

dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg.jpg'

In [19]:
from graphviz import Digraph

dot = Digraph(comment='Rectangle with Dividing Line')

dot.node('A', shape='record', label='{Above Text|Below Text}')

dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg'

In [225]:
import graphviz as gv
dot = gv.Digraph(comment='Flowchart')

In [226]:
dummyCounter = 0

# mmaybe pass ID of activity, semaphore that you want to color for the animation

def createRects():
    for activity in activities:
        dot.node(str(activity.ID), shape='record', style='rounded', label='{'+activity.parentTask.name+'|'+activity.name+'}')
        
def createMutexs():
    for mutex in mutexs:
        dot.node(str(mutex.ID), shape='polygon', sides='5', label=mutex.name)
        for activity in mutex.activities:
            dot.edge(str(mutex.ID), str(activity.ID), arrowhead='none', style='dashed', splines='polyline')
        
def createSemaphores():
    global dummyCounter
    
    semaphores_buf = semaphores.copy()
    while len(semaphores_buf) != 0:
        semaphore = semaphores_buf.pop(0)
        # if semaphore has groupWith, connect those semaphores together. Those will then be erased from the list
        if len(semaphore.groupWith) != 0:
            dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01")
            middleDummy = "Dummy" + str(dummyCounter)
            
            # first the semaphore itself
            dot.edge(str(semaphore.activityOUT.ID), middleDummy, label=semaphore.name, arrowhead='none', splines='polyline')
            dot.edge(middleDummy, str(semaphore.activityIN.ID), arrowhead='normal', splines='polyline')
            dummyCounter += 1
            
            # after the groupSemaphores
            for groupSemaphore in semaphore.groupWith:
                dot.edge(str(groupSemaphore.activityOUT.ID), middleDummy, label=groupSemaphore.name, arrowhead='none', splines='polyline')
                semaphores_buf.remove(groupSemaphore)
        else:
            # if semaphore is leading to the same task where it came from, use another arrowhead
            if semaphore.activityIN.parentTask.ID == semaphore.activityOUT.parentTask.ID:
                # if semaphore has an initial value, connect an edge to the semaphore
                if not semaphore.initialValue == "None":
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01")
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dot.edge("Dummy" + str(dummyCounter), str(semaphore.activityIN.ID), label=semaphore.name, arrowhead='onormal', splines='polyline')
                    dummyCounter += 1
                    dot.node("Dummy" + str(dummyCounter), shape='point')
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(semaphore.activityOUT.ID), str(semaphore.activityIN.ID), label=semaphore.name, arrowhead='onormal', splines='polyline')
            else:
                if not semaphore.initialValue == "None":
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01")
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dot.edge("Dummy" + str(dummyCounter), str(semaphore.activityIN.ID), label=semaphore.name, splines='polyline')
                    dummyCounter += 1
                    dot.node("Dummy" + str(dummyCounter), shape='point')
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(semaphore.activityOUT.ID), str(semaphore.activityIN.ID), label=semaphore.name, splines='polyline')
            

In [227]:
createRects()
createMutexs()
createSemaphores()
dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg'